# Aula 5 - Solução dos exercícios - Parte 3
Leandro Carísio Fernandes

<br>

Treinar um modelo seq2seq (a partir do T5-base) na tarefa de expansão de documentos.

- Usar como treino o dataset "tiny" do MS MARCO na tarefa doc2query
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
- doc2query: A entrada é a passagem e o target é a query
- Note que apenas pares (query, passagem relevante) são usados como treino.
- O treino é relativamente rápido (<1 hora).
- Validar a cada X steps usando o sacreBLEU 
- A parte lenta deste exercício é a pré-indexação: para cada documento da coleção, temos que gerar uma ou mais queries, que depois são concatenadas ao documento original, e esse documento "expandido" é indexado.
- Avaliar no TREC-COVID (171K docs), pois é menor que o MS MARCO/TREC-DL 2020 (8.8M passagens). 
 - Indice invertido do Trec-covid no pyserini: beir-v1.0.0-trec-covid-flat
 - Corpus e queries na HF: https://huggingface.co/datasets/BeIR/trec-covid
 - qrels: https://huggingface.co/datasets/BeIR/trec-covid-qrels
 - Usar nDCG@10
 - Comparar com o BM25 com e sem os documentos expandidos pelo doc2query


<br>

Esse caderno contém apenas a parte 3 do exercício, a implementação do BM25 com expansão de query.

## Preparação do ambiente

### Variáveis usadas para controlar o fluxo do caderno

In [ ]:
# Forçar refazer o índice invertido
forcar_recalcular_indice_trec_covid = True

# Link de download da base TREC-COVID. O link foi retirado de https://huggingface.co/datasets/BeIR/trec-covid
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

# Arquivo gerado com as queries expandidas
arquivo_docs_queries_expandidas = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/doc_com_queries_expandidas.pickle'

### Instalação de libs e montagem do Drive

In [ ]:
%%time
# Já monta o drive, pois vamos usar o índice invertido da Aula 1 para usar o BM25 implementado também na aula 1
# Além disso, é necessário para salvar/recuperar o modelo tunado
from google.colab import drive
drive.mount('/content/drive')

# Instala libs
!pip install pyserini
!pip install faiss-cpu

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.5 MB/s eta 0:0

### Download do TREC-COVID

In [ ]:
from pathlib import Path
import hashlib

# Baixa o trec-covid.zip
if not Path('./collections/trec-covid.zip').is_file():
  # O arquivo ainda não foi baixado. Verifica se está no drive:
  if Path('/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/trec-covid.zip').is_file():
    !mkdir -p './collections/' && cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/trec-covid.zip' './collections/trec-covid.zip'  # type: ignore
    !unzip -o collections/trec-covid.zip -d ./collections # type: ignore
  else:
    !wget {url_trec_covid} -P collections # type: ignore
    !mkdir -p '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/' && cp './collections/trec-covid.zip' '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/'
    !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

# Converte o qrels que veio no trec-covid.zip pra um formato mais adequado:
with open('./collections/trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)
with open('./collections/trec-covid/qrels/test_corrigido.tsv', 'w') as fout:
  for linha in data[1:]:
    campos = linha.split()
    fout.write(f'{campos[0]}\t0\t{campos[1]}\t{campos[2]}\n')

Archive:  collections/trec-covid.zip
   creating: ./collections/trec-covid/
   creating: ./collections/trec-covid/qrels/
  inflating: ./collections/trec-covid/qrels/test.tsv  
  inflating: ./collections/trec-covid/corpus.jsonl  
  inflating: ./collections/trec-covid/queries.jsonl  


### Classes para índice invertido e pesquisa com BM25

Nessa seção vou usar as implementações de índice invertido e BM25 que eu já havia feito na Aula 2.

Entretanto, algumas adaptações foram necessárias. Por exemplo:

- A ID dos documentos na avaliação anterior era um inteiro, e isso era armazenado na classe IndiceInvertido como um array.array. No TREC_COVID a ID é uma string, então a classe teve que ser alterada para usar uma lista em vez de array.array.
- A classe de pesquisa do BM25 antes calculava o score tanto pelo BM25 quanto pelo TF/IDF. Removi essa última parte pois não é relevante para esse exercício.


Implementação das classes IndiceInvertido e BM25.

In [ ]:
%%time
from pyserini.analysis import Analyzer, get_lucene_analyzer
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertido:
  lista_stopwords_nltk = set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.lucene_analyser = Analyzer(get_lucene_analyzer(stemmer='porter'))

  def tokenizar(self, texto):
    return self.lucene_analyser.analyze(texto)

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)
    
    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)
  
  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])   
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']
      
      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 555 ms, sys: 71.3 ms, total: 626 ms
Wall time: 1.04 s


### Funções para carregar e processar o corpus TREC_COVID e carregar as queries e qrels

In [ ]:
from collections import OrderedDict
trec_covid_docs = OrderedDict({})
with open(arquivo_docs_queries_expandidas, 'rb') as f:
  trec_covid_docs = pickle.load(f)

In [ ]:
%%time
import json

def carrega_queries_trec_covid():
  retorno = []
  with open('./collections/trec-covid/queries.jsonl') as queries:
    for line in queries:
      query = json.loads(line)
      # Faz apenas uma pequena tradução de _id para id e text para texto
      retorno.append({'id': query['_id'], 'texto': query['text']})
  return retorno

queries_trec_covid = carrega_queries_trec_covid()

CPU times: user 594 µs, sys: 0 ns, total: 594 µs
Wall time: 604 µs


### Função para executar as queries em um buscador

In [ ]:
# Roda todas as queries
def run_all_queries(file, buscador_bm25):
  print('Carregando as queries do arquivo queries.jsonl...\n')
  queries_trec_covid = carrega_queries_trec_covid()

  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id = query['id']
      texto = query['texto']

      if cnt % 10 == 0:
        print(f'{cnt} queries completadas')

      # Usa o BM25 para buscar
      docs_score = buscador_bm25.pesquisar(texto)

      for i in range(0, min(1000, len(docs_score))): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))

      cnt += 1
    print(f'{cnt} queries completadas')

## Análise do BM25 na base TREC-COVID

### Carregando o corpus.jsonl no índice invertido

O arquivo corpus.jsonl possui um documento por linha, sendo que todos possuem o seguinte formato:

    {"_id": ..., "title": ..., "metadata": ...}

Nos interessa aqui apenas o title e o _id. Vamos criar o índice invertido:

In [ ]:
%%time
def salvar_indice_em_arquivo_pickle(indice_invertido, nome_arquivo):
  with open(f'{nome_arquivo}_indice.pickle', 'wb') as f:
    pickle.dump(indice_invertido.indice, f)
  with open(f'{nome_arquivo}_tamanho_doc.pickle', 'wb') as f:
    pickle.dump(indice_invertido.tamanho_doc, f)

def montar_indice_a_partir_arquivo_pickle(nome_arquivo):
  idx = IndiceInvertido()

  with open(f'{nome_arquivo}_indice.pickle', 'rb') as f:
    idx.indice = pickle.load(f)
  with open(f'{nome_arquivo}_tamanho_doc.pickle', 'rb') as f:
    idx.tamanho_doc = pickle.load(f)
  idx.n_docs = len(idx.tamanho_doc)

  return idx

def criar_indice_invertido_trec_covid():
  iidx_trec_covid = IndiceInvertido()

  # Carrega todo o corpus no índice invertido
  for i, id_doc in enumerate(trec_covid_docs.keys()):
    if 'query_expandida' in trec_covid_docs[id_doc]:
      iidx_trec_covid.adiciona_doc(id_doc, f"{trec_covid_docs[id_doc]['doc_original']} {trec_covid_docs[id_doc]['query_expandida']}")
    else:
      iidx_trec_covid.adiciona_doc(id_doc, f"{trec_covid_docs[id_doc]['doc_original']}")

    if i % 20000 == 0:
      print(f'{i} queries processadas')

  # Salva no drive a última versão do índice
  salvar_indice_em_arquivo_pickle(iidx_trec_covid, 'iidx_trec_covid_com_expansao_queries')

  # Copia arquivos de índice e tamanho do documento para o google drive:
  !cp iidx_trec_covid_com_expansao_queries*.pickle '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/'  # type: ignore
  
  return iidx_trec_covid

def copiar_indice_invertido_do_drive():
  iidx_trec_covid = IndiceInvertido()

  if Path('/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/iidx_trec_covid_com_expansao_queries_indice.pickle').is_file():
    print('Copiando índice do drive...')
    !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/iidx_trec_covid_com_expansao_queries_indice.pickle' './iidx_trec_covid_com_expansao_queries_indice.pickle'  # type: ignore
    !cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/iidx_trec_covid_com_expansao_queries_tamanho_doc.pickle' './iidx_trec_covid_com_expansao_queries_tamanho_doc.pickle'  # type: ignore
    
    iidx_trec_covid = montar_indice_a_partir_arquivo_pickle('iidx_trec_covid_com_expansao_queries')

  else:
    iidx_trec_covid = criar_indice_invertido_trec_covid()

  return iidx_trec_covid

iidx_trec_covid = criar_indice_invertido_trec_covid() if forcar_recalcular_indice_trec_covid else copiar_indice_invertido_do_drive()

0 queries processadas
20000 queries processadas
40000 queries processadas
60000 queries processadas
80000 queries processadas
100000 queries processadas
120000 queries processadas
140000 queries processadas
160000 queries processadas
CPU times: user 2min 17s, sys: 1.54 s, total: 2min 19s
Wall time: 2min 36s


### Avalia o nDCG@10 nas 50 queries disponíveis

Nota: Vou avaliar duas vezes o qrels, uma no arquivo que está no trec-covid.zip e outra no arquivo equivalente do repositório do Anserini. Ambos os arquivos são equivalentes, exceto por duas entradas com score negativo que existem no trec-covid.zip que não existe no Anserini:

- 38	9hbib8b3	-1
- 50	svo94kuo	-1


In [ ]:
%%time
#buscador = BM25(iidx_trec_covid, 0.9, 0.4, 1)
buscador = BM25(iidx_trec_covid, 0.82, 0.68, 1)
run_all_queries('run-bm25-com-expansao-query-trec-covid.txt', buscador)

print('Avaliando nDCG usando o qrels que está no HugingFace:\n')
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-bm25-com-expansao-query-trec-covid.txt #type: ignore

#print('\nAvaliando nDCG usando o qrels que está no Anserini:\n')
#!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/qrels.beir-v1.0.0-trec-covid.test.txt run-bm25-sem-expansao-query-trec-covid.txt #type: ignore

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
Avaliando nDCG usando o qrels que está no HugingFace:

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 28.6MB/s]
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-bm25-com-expansao-query-trec-covid.txt']
Results:
ndcg_cut_10           	all	0.6719
CPU times: user 25.5 s, sys: 287 ms, total: 25.8 s
Wall time: 43.4 s


In [ ]:
%%time
# Faz uma pesquisa geral de melhores parâmetros k1 e b:

for i in range (80, 95, 2):
  k1 = i/100
  for j in range(60, 80, 2):
    b = j/100

    buscador = BM25(iidx_trec_covid, k1, b, 1)
    print('**********************************************')
    print(f'Resultados de nDCG@10 para k1 = {k1} e b = {b}')
    run_all_queries('teste.txt', buscador)
    !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv teste.txt #type: ignore

**********************************************
Resultados de nDCG@10 para k1 = 0.8 e b = 0.6
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'teste.txt']
Results:
ndcg_cut_10           	all	0.6710
**********************************************
Resultados de nDCG@10 para k1 = 0.8 e b = 0.62
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cach

In [ ]:
%%time
# Faz uma pesquisa geral de melhores parâmetros k1 e b:

for i in range (96, 111, 2):
  k1 = i/100
  for j in range(60, 80, 2):
    b = j/100

    buscador = BM25(iidx_trec_covid, k1, b, 1)
    print('**********************************************')
    print(f'Resultados de nDCG@10 para k1 = {k1} e b = {b}')
    run_all_queries('teste.txt', buscador)
    !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv teste.txt #type: ignore

**********************************************
Resultados de nDCG@10 para k1 = 0.96 e b = 0.6
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'teste.txt']
Results:
ndcg_cut_10           	all	0.6852
**********************************************
Resultados de nDCG@10 para k1 = 0.96 e b = 0.62
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.ca

In [ ]:
%%time
# Faz uma pesquisa geral de melhores parâmetros k1 e b:

for i in range (106, 121, 2):
  k1 = i/100
  for j in range(50, 60, 2):
    b = j/100

    buscador = BM25(iidx_trec_covid, k1, b, 1)
    print('**********************************************')
    print(f'Resultados de nDCG@10 para k1 = {k1} e b = {b}')
    run_all_queries('teste.txt', buscador)
    !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv teste.txt #type: ignore

**********************************************
Resultados de nDCG@10 para k1 = 1.06 e b = 0.5
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'teste.txt']
Results:
ndcg_cut_10           	all	0.6988
**********************************************
Resultados de nDCG@10 para k1 = 1.06 e b = 0.52
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.ca

In [ ]:
%%time
# Faz uma pesquisa geral de melhores parâmetros k1 e b:

for i in range (110, 121, 2):
  k1 = i/100
  for j in range(40, 50, 2):
    b = j/100

    buscador = BM25(iidx_trec_covid, k1, b, 1)
    print('**********************************************')
    print(f'Resultados de nDCG@10 para k1 = {k1} e b = {b}')
    run_all_queries('teste.txt', buscador)
    !python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv teste.txt #type: ignore

**********************************************
Resultados de nDCG@10 para k1 = 1.1 e b = 0.4
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'teste.txt']
Results:
ndcg_cut_10           	all	0.7001
**********************************************
Resultados de nDCG@10 para k1 = 1.1 e b = 0.42
Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cach